# Creative machine learning - Neural networks

### Author: Philippe Esling (esling@ircam.fr)

In this course we will cover
1. A [quick introduction](#intro) on the principles of neural networks
2. An implementation for a [single neuron](#neuron) in Numpy and JAX.
3. An exercise on [multi-layer perceptron (MLP)](#mlp) through manual derivation.
4. An introduction on [using Pytorch](#pytorch) for defining networks
5. An exercise on [audio classification](#audio) using an MLP with Pytorch.

<a id="intro"></a>
# Introducing neural networks

In this tutorial, we will cover more advanced models known as *neural networks*. The tutorial starts by performing a simple **single neuron** discrimination of two random distributions. We will exhibit the manual implementation using Numpy, and then simplify it with JAX. Then, we will study the typical **XOR problem** by using a more advanced 2-layer **perceptron**. Finally, we generalize the use of neural networks in order to perform classification on a given set of audio files, using the PyTorch library, which will provide simplified implementation.

We are going to use relatively _low-level_ libraries to perform the first exercises (implementing your own neurons). To observe this idea in simple setups, we are going to use the `numpy` library and also initialize the homemade course library `cml` and style for future plotting and exercise. We also set the random generator to a fixed point with `rng = np.random.RandomState(1)`, to ensure reproducibility

In [1]:
# Base imports
import time
import numpy as np
import matplotlib.pyplot as plt
from cml.plot import initialize_bokeh
from cml.panel import initialize_panel
from jupyterthemes.stylefx import set_nb_theme
from bokeh.io import show
initialize_bokeh()
initialize_panel()
set_nb_theme("onedork")
rng = np.random.RandomState(1)

2024-04-25 15:25:32.932136: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 15:25:33.923709: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Loading BokehJS ...

As usual, to simplify your work, we provide sets of functions that allows fast problem definition and plotting functionnalities (from our `cml` library).

  |**File**|*Explanation*|
  |-------:|:---------|
  |`scatter_boundary`|Plots the decision boundary of a single neuron with 2-dimensional inputs|
  |`scatter_classes`|Plots (bi-dimensionnal) input patterns|

In [2]:
from cml.plot import scatter_classes, scatter_boundary, center_plot

Remember that a single neuron is only able to learn _linearly separable_ problems. To produce such classes of problems, we provide a script that draw a set of random 2-dimensional points, then choose a random line in this space that will act as the linear frontier between 2 classes (hence defining a linear 2-class problem). The variables that will be used by your code are the following.  

```Python
y_class       # classes of the observqtions 
x_inputs      # 2 x n final matrix of random input observations
weights       # 2 x 1 vector of neuron weights
bias          # 1 x 1 vector of bias
```

You can execute the code below to see our simple classification problem. (Note that running the same cell multiple times produces a different starting dataset). In order to have a well-defined classification problem, we can rely on the `make_blobs` function provided by `scikit-learn` in the `sklearn.datasets` package.

In [4]:
import numpy as np
from sklearn.datasets import make_blobs
# Properties of the problem
n_observations = 200
noise = 0.2
c1_center = [-2, -1]
c2_center = [2, 1]
# Create points
x_coords, y_class = make_blobs(n_samples=n_observations, centers=[c1_center, c2_center], n_features=2, cluster_std=0.45)
x_inputs = x_coords + (noise * np.random.randn(n_observations, 2))
# Plot the corresponding pattern
plot = center_plot(scatter_classes(x_inputs, y_class))
plot.show()

Launching server at http://localhost:42621


The previous example does not allow us to know the exact values (slope and bias) of our ground truth separation line. Hence, we can define a more complex separation problem (with points laying almost right on the separation fronteer), but with known values for our ground truth.

In [8]:
# Number of points to generate
n_observations = 100;
# Generate 2-dimensional random points
x_inputs = np.random.rand(int(n_observations), 2) * 2 - 1;
# Slope of separating line
sep_slope = np.log(np.random.rand() * 10);
sep_bias = np.random.rand() * 2 - 1;
# Create the indexes for a two-class problem
y_class = (x_inputs[:, 1] - x_inputs[:, 0] * sep_slope - sep_bias > 0) * 1;
# Plot the corresponding pattern
plot = center_plot(scatter_classes(x_inputs, y_class))
print(sep_slope)
print(sep_bias)
plot

2.2517524985069075
-0.3536943653964144


BokehModel(combine_events=True, render_bundle={'docs_json': {'c42b8ae3-0ca8-4bb3-b758-3b9c174eceee': {'version…

<a id="neuron"></a>
## Single neuron

For the first parts of the tutorial, we will perform the simplest classification model possible in a neural network setting, a single neuron. We briefly recall here that; given an input vector $\mathbf{x} \in \mathbb{R}^{n}$, a single neuron computes the function  

$$
\begin{equation}
\bar{y}=\phi\left(\sum_{i = 1}^{n}w_{i}.x_{i} + b\right)
\label{eq1}
\end{equation}
$$

with $ \mathbf{w} \in \mathbb{R}^{n} $ a weight vector, $ b $ a bias and $ \phi\left( \cdot \right) $ an *activation function*. Therefore, if we consider the *threshold* activation function ($ \mathbb{I}_{0}\left(x\right)=1 $ if $ x \geq 0$), a single neuron simply performs an *affine transform* and then a *linear* discrimination of the space. 

***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #410819; border-color: #cb2e47">

> **Important note**
> The following implementation tries to remain as close as possible to the original neuron model by McCulloch & Pitts, notably by using the _threshold activation_ function. Although this leads to a very simple implementation (akin to using an _identity_ activation, note that it is not to be used afterwards.

</div>

***

As we will see later on, a **neural network** will simply be composed of _layers_ of these neurons, which produce successive computations

<img src="images/02_feedforward_nn.png" align="center"/>

Geometrically, a single neuron computes an hyperplane that separates the space. In order to learn, we have to adjust the weights and know "how much wrong we are". To do so, we consider that we know the desired output $ y_{j} $ of a system for a given example $ \mathbf{x}_{j} $ (eg. a predicted value for a regression system, a class value for a classification system). Therefore, we define the MSE loss function $ \mathcal{L}_{\mathcal{D}} $ over a whole dataset as

$$
\begin{equation}
\mathcal{L}_{\mathcal{D}}=\sum_{j=1}^{|\mathcal{D}|}\left\Vert \bar{y}_{j}-y_{j}\right\Vert ^{2}
\label{eq2}
\end{equation}
$$

with $|\mathcal{D}|$ being the size of our dataset. In order to know how to change the weights based on the value of the errors, we need to now "how to change it to make it better". Therefore, we should compute the sets of derivatives of the error given each parameter

$$
\begin{equation}
\Delta\bar{\mathbf{w}}=\left(\frac{\delta\mathcal{L}_{\mathcal{D}}}{\delta w_{1}},\ldots,\frac{\delta\mathcal{L}_{\mathcal{D}}}{\delta w_{n}}\right)
\label{eq3}
\end{equation}
$$ 

***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Exercise (course)
>   1. Perform the derivatives of the output given a single neuron
>   2. Perform the derivatives for the bias as well

</div>

***

### Training your own neuron

We will start by training a single neuron to learn how to perform this discrimination with a linear problem (so that a single neuron is enough to solve it). 

In [5]:
# Learning rate
eta = 1e-2;
# Weight decay
lambda_r = 0.1
# Number of epochs
n_epochs = 50
# Initialize the weights
weights = np.random.randn(1, 2);
bias = np.random.randn(1, 1);
# Save the weight history for plotting
weights_history = np.zeros((n_epochs + 1, 2));
bias_history = np.zeros((n_epochs + 1, 1));

Now you need to update the following code loop to ensure that your neuron learns to separate between the classes 
***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Exercise (course)
>   1. Update the loop so that it computes the forward propagation error
>   2. Update the loop to perform learning (based on back-propagation)
>   3. Run the learning procedure, which should produce a result similar to that displayed on the website
>   4. Perform multiple re-runs by **tweaking the hyperparameters** (learning rate, weight decay)
>   5. What observations can you make on the learning process?
>   6. (Optional) Change the input patterns, and confirm your observations.
>   6. (Optional) Incorporate the bias in the weights to obtain a **vectorized** code.

</div>

***

In [9]:
verbose = False
weights_history[0] = weights
bias_history[0] = bias
# Update loop
for i in range(n_epochs):
    
    ######################
    # YOUR CODE GOES HERE
    ######################
    
    ######################
    # Solution  
    y_bar = np.dot(weights, x_inputs.T) + bias > 0
    if np.sum(np.abs(y_bar - y_class)) == 0:
        break
    error = y_class - y_bar
    weights = weights + eta * (np.dot(error, x_inputs))
    bias = bias + eta * (np.sum(error))
    ######################
    
    weights_history[i + 1] = weights
    bias_history[i + 1] = bias
    
    if (verbose):
        print('%2d. error = %f, weights = %f, %f, %f' % (i, np.sum(np.abs(error)) / n_observations, bias[0, 0], weights[0, 0], weights[0, 1]))
plot = center_plot(scatter_boundary(x_inputs, y_class, weights_history, bias_history, i + 1))
plot.show()

Launching server at http://localhost:41135


Opening in existing browser session.


### Neuron implementation in `JAX`

As seen in the previous course, the `NumPy` implementation requires us to perform manual differentiation of our loss function to understand how to update the parameters. However, the recent [`JAX`](https://github.com/google/jax) library provides **automatic differentiation** on top of the `NumPy` library. If you have not yet completed the set of [tutorials](https://jax.readthedocs.io/en/latest/), we strongly encourage you to do so. We will introduce a few more functions compared to last time, but we try to keep the implementation as simple as possible on purpose.

In [10]:
import jax.numpy as jnp
from jax import grad, jit, random

seed = np.random.randint(0, 1e9)
np.random.seed(23)

In [47]:
x_jax = jnp.asarray(x_inputs)
y_jax = jnp.asarray(y_class)

We will also keep the same hyperparameters as previously (see `eta`, `lambda_r` and `n_epochs` above) and randomly initialize our parameters 

In [48]:
eta = 0.01
# Perform key splitting
key_w, key_b, key = random.split(key, 3)
# Initialize the weights
weights = random.normal(key_w, (1, 2));
bias = random.normal(key_b, (1, 1));
# Save the weight history for plotting
weights_history = jnp.zeros((n_epochs + 1, 2));
bias_history = jnp.zeros((n_epochs + 1, 1));
# Record the first 
weights_history = weights_history.at[0].set(weights[0])
bias_history = bias_history.at[0].set(bias[0])

As we discussed earlier, the original model by McCulloch & Pitts has several issues regarding its use for numerical optimization (as it was originally observed from a neuroscience point of view)

***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #410819; border-color: #cb2e47">

> **Changes from the `NumPy` implementation**
>   1. (Smoothness) Our _activation function_ will change from _identity_ to sigmoid, i.e. $ \phi(\mathbf{x}) = \frac{1}{1 + e^{-\mathbf{x}}} $
>   2. (Outliers) Our _loss function_ will change from $ L_{1} $ to $ L_{2} $ (MSE - Mean Squared Error)

</div>

***

As discussed in the previous course, for using the very handy `grad` function provided by `JAX`, we need to define the whole forward function that we want to derive given our parameters. Hence this function also needs to **contain the forward pass**. This is summarized in the following function

In [49]:
import jax
# Single neuron full function
def loss_function(x, y, w, b):
    # Forward pass
    y_bar = jax.nn.sigmoid(jnp.dot(w, x.T) + b)
    # Error computation
    error = jnp.sum((y_bar - y) ** 2)
    print(error)
    return error

grad_FUNCTION = grad(loss_function, argnums=[2, 3])
dW, dB = grad_FUNCTION(x, y, w, b)
# Gradient of the loss
grad_loss_function = grad(loss_function, argnums=[2, 3])

Finally, we can now define the main training loop, which is almost exactly similar to the previously defined one minus the gradient computation operations. 

In [50]:
# Update loop
for i in range(n_epochs):
    
    ######################
    # YOUR CODE GOES HERE
    ######################
    
    ######################
    # Solution 
    gradients = grad_loss_function(x_jax, y_jax, weights, bias)
    weights = weights - eta * gradients[0]
    bias = bias - eta * gradients[1]
    ######################
    weights_history = weights_history.at[i + 1].set(weights[0])
    bias_history = bias_history.at[i + 1].set(bias[0])
    
    if (verbose):
        print('%2d. error = %f, weights = %f, %f, %f' % (i, np.sum(np.abs(error)) / n_observations, bias[0, 0], weights[0, 0], weights[0, 1]))
plot = center_plot(
    scatter_boundary(
        np.array(x_inputs), 
        np.array(y_class), 
        np.array(weights_history), 
        np.array(bias_history), 
        i + 1)
    )
plot

Row
    [0] HSpacer()
    [1] Bokeh(Figure)
    [2] HSpacer()

## Multi-layer networks

In the following, we define the overall architecture of the exercise to fill. The goal is to define a more advanced problem 

### 2-layer XOR problem

In most cases, classification problems are far from being linear. Therefore, we need more advanced methods to be able to compute non-linear class boundaries. The advantage of neural networks is that the same principle can be applied in a *layer-wise* fashion. This allows to further discriminate the space in sub-regions (as seen in the course). We will try to implement the 2-layer *perceptron* that can provide a solution to the infamous XOR problem. The idea is now to have the output of the first neurons to be connected to a set of other neurons. Therefore, if we take back our previous formulation, we have the same output for the first neuron(s) $y$, that we will now term as $y^{(1)}$. Then, we feed these outputs to a second layer of neurons, which gives


$$
\begin{equation}
y^{(2)}=\sigma\left(\sum_{i = 1}^{n}w_{i}.y^{(1)}_{i} + b\right)
\end{equation}
$$  

Finally, we will rely on the same loss $\mathcal{L_{D}}$ as in the previous exercise, but the outputs used are $y^{(2)}$ instead of $y$. As in the previous case, we now need to compute the derivatives of the weights and biases for several layers . However, you should see that some form of generalization might be possible for any number of layer.  

***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Exercise (course)
>   1. Perform the derivatives for the last layer specifically
>   2. Define a generalized derivative for any previous layer

</div>

***

We can construct the prototypical set of XOR values by using the following code (note that this is the most simple case, but still this is typically a problem that cannot be solved by a _linear classifier_

In [54]:
# Input patterns
x_inputs = np.array([[-1, -1],[-1,  1],[1, -1],[1,  1]])
x_inputs + (np.random.randn(()) * 0.1)
# Corresponding classes
y_class = np.array([0, 1, 1, 0])                        
# Initialize based on their sizes
n_inputs = x_inputs.shape[0]
n_outputs = 1
# Plot the XOR problem
plot = center_plot(scatter_classes(x_inputs, y_class, title = "XOR classification"))
plot

Row
    [0] HSpacer()
    [1] Bokeh(figure)
    [2] HSpacer()

## Observing the problem interactively

In [ ]:
from cml.tasks import ClassificationLinear, ClassificationXOR
explorer = ClassificationXOR()
explorer.render()

Creating the object


Column
    [0] Row
        [0] Param(ClassificationXOR, name='Problem parameters')
        [1] ParamMethod(method, _pane=Bokeh)
    [1] CodeEditor(height=60, language='python', theme='chaos', value='    def generate_data(sel...)

## Solving the problem with the `scikit-learn` library

As a side note, and as discussed in the previous course, there is obviously a lot of libraries that can perform the training of such networks for you directly, without understanding what is exactly going on behind the scene. A quite extensive library is `scikit-learn`, which contains already coded models and learning procedure, that will allow us to _learn_ the parameters of this unknown function.

Here we will use a `MLPClassifier` model to perfom classification on standardized features that we rescale through the `LinearRegression` and that this polynomial should be of degree 2. 

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
# Create the MLP classifier
classifier = MLPClassifier(alpha=1, max_iter=1000)
# Standardize the input features
clf = make_pipeline(StandardScaler(), classifier)

Now based on this model definition, training it can be performed with a single line of code

In [7]:
from cml.data import xor_separation
# Generate a XOR dataset
x_inputs, y_classes = xor_separation(500, 0.1)
# Train the MLP model
clf.fit(x_inputs, y_classes);

## Multi-layer implementation

In this exercise, we will implement the XOR classification resolution using NumPy and JAX. This classification problem cannot be solved with a simple linear classifier. Hence, we will need to use at least a 2-layer network. The minimal set of variables that will be used by your code should be the following.

```Python
x_inputs          # 2 x n matrix of random points
y_class           # classes of the patterns 
n_hidden          # Number of hidden units
eta               # Learning rate parameter
momentum          # Momentum parameter (bonus)
weights1          # 1st layer weights
weights2          # 2nd layer weights
mse_limit         # Sum-squared error limit
```

In [8]:
# Number of hidden units
n_hidden = 2
# Learning rate parameter
eta = 0.005
# Momentum parameter
momentum = 0.1
# Sum-squared error limit
mse_limit = 0.002

## Exercise 1 - 2-layers XOR classification

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> To complete this exercise, you will need to have a basic understanding of NumPy and JAX. You can use the resources provided in the previous exercises to learn more about these libraries.

We help you out by first defining the XOR classification problem to solve (and outlining some basic properties)

</div>

In [9]:
from cml.data import xor_separation
# Definition of our data properties
n_observations = 500
noise_factor = 0.1
# Properties of our problem
n_input = 2
n_output = 1
# Generate a XOR dataset
x_inputs, y_classes = xor_separation(n_observations, noise_factor)

We also show some potential implementation of the **initialization scheme**. Note that you can later try to replace this naive implementation by some more refined functions

In [10]:
import numpy as np
# 1st layer weights
weights1 = (np.random.randn(n_hidden, n_input) * 0.01)
# 2nd layer weights
weights2 = (np.random.randn(n_output, n_hidden) * 0.01)
# 1st layer biases
bias1 = (np.random.randn(n_hidden, 1) * 0.01)
# 2nd layer biases
bias2 = (np.random.randn(n_output, 1) * 0.01)

***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.1 - Implement 2-layer XOR classification with NumPy

> 1. Update the forward propagation and error computation
> 2. Update the back-propagation part to learn the weights of both layers.
> 3. Run the learning and check your network results
  
*For optional questions, please look at the end of this exercise's code boxes for more information*

</div>

***

In [60]:

######################
# YOUR CODE GOES HERE
######################

from tqdm import tqdm
import numpy as np
weights1 = (np.random.randn(n_hidden, n_input))
weights2 = (np.random.randn(n_output, n_hidden))
bias1 = (np.random.randn(n_hidden, 1))
bias2 = (np.random.randn(n_output, 1))

# weights1 = (np.zeros((n_hidden, n_input)))
# weights2 = (np.zeros((n_output, n_hidden)))
# bias1 = (np.zeros((n_hidden, 1)))
# bias2 = (np.zeros((n_output, 1)))

n_epochs = 1000
p_bar = tqdm(range(n_epochs))
for i in p_bar:
    
    z1 = np.dot(weights1, x_inputs.T) + bias1
    a1 = 1 / (1 + np.exp(-z1))
    z2 = np.dot(weights2, a1) + bias2
    a2 = 1 / (1 + np.exp(-z2))

    # Error computation
    error = np.sum((a2 - y_classes) ** 2)
    if error < mse_limit:
        break

    # Backward pass
    delta2 = (a2 - y_classes) * a2 * (1 - a2)
    delta1 = np.dot(weights2.T, delta2) * a1 * (1 - a1)

    # Update weights
    weights2 = weights2 - eta * np.dot(delta2, a1.T)
    bias2 = bias2 - eta * np.sum(delta2, axis = 1, keepdims = True)
    weights1 = weights1 - eta * np.dot(delta1, x_inputs)
    bias1 = bias1 - eta * np.sum(delta1, axis = 1, keepdims = True)
    
    p_bar.set_description(f"Error: {error:.4f}")

y_pred = np.round(a2)[0]

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create figure

fig = make_subplots(rows=1, cols=2, subplot_titles=("XOR classification", "XOR classification"))
# Plot the original XOR problem
fig.add_trace(go.Scatter(x=x_inputs[y_classes == 0, 0], y=x_inputs[y_classes == 0, 1], mode='markers', name='Class 0', marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=x_inputs[y_classes == 1, 0], y=x_inputs[y_classes == 1, 1], mode='markers', name='Class 1', marker=dict(color='red')), row=1, col=1)
# Plot the XOR classification
fig.add_trace(go.Scatter(x=x_inputs[y_pred == 0, 0], y=x_inputs[y_pred == 0, 1], mode='markers', name='Class 0', marker=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=x_inputs[y_pred == 1, 0], y=x_inputs[y_pred == 1, 1], mode='markers', name='Class 1', marker=dict(color='red')), row=1, col=2)

# Add the decision boundary
x1 = np.linspace(-1.5, 1.5, 100)
x2 = np.linspace(-1.5, 1.5, 100)
X1, X2 = np.meshgrid(x1, x2)
X = np.array([X1.flatten(), X2.flatten()]).T
Z1 = 1 / (1 + np.exp(-(np.dot(weights1, X.T) + bias1)))
Z2 = 1 / (1 + np.exp(-(np.dot(weights2, Z1) + bias2)))
Z = Z2.reshape(100, 100)

fig.add_trace(go.Contour(x=x1, y=x2, z=Z, colorscale='Viridis', showscale=False), row=1, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="X1", row=1, col=1)
fig.update_xaxes(title_text="X1", row=1, col=2)
# Update yaxis properties
fig.update_yaxes(title_text="X2", row=1, col=1)
fig.update_yaxes(title_text="X2", row=1, col=2)
# Update title and size
fig.show()

Error: 124.0000:  29%|██▉       | 294/1000 [00:00<00:00, 970.03it/s]

Error: 124.0000: 100%|██████████| 1000/1000 [00:01<00:00, 835.59it/s]


***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.2 - Implement XOR classification with JAX

> 1. Perform the same implementation as previously with JAX
> 2. Implement gradient descent to optimize the weights and bias.

</div>

***

In [67]:

######################
# YOUR CODE GOES HERE
######################

from tqdm import tqdm
from jax import grad, jit, random
import jax.numpy as jnp

key = random.PRNGKey(np.random.randint(0, 1e9))
keys = random.split(key, 4)

def xavier_init(shape):
    return random.normal(keys[0], shape) * jnp.sqrt(2. / (shape[0] + shape[1]))

# Different initialization for better results
# weights1 = xavier_init((n_hidden, n_input))
# weights2 = xavier_init((n_output, n_hidden))
weights1 = random.normal(keys[0], (n_hidden, n_input))
weights2 = random.normal(keys[1], (n_output, n_hidden))
bias1 = random.normal(keys[2], (1, n_hidden))
bias2 = random.normal(keys[3], (1, n_output))

@jit
def mlp(x, w1, w2, b1, b2):
	z1 = jnp.dot(w1, x.T) + b1.reshape(-1, 1)
	# a1 = jnp.maximum(0, z1)
	a1 = 1 / (1 + jnp.exp(-z1))
	z2 = jnp.dot(w2, a1) + b2.reshape(-1, 1)
	# a2 = jnp.maximum(0, z2)
	a2 = 1 / (1 + jnp.exp(-z2))
	return a2[0]

@jit
def loss_function(x, y, w1, w2, b1, b2):
	# Forward pass
	y_bar = mlp(x, w1, w2, b1, b2)
	# Error computation
	error = jnp.sum((y_bar - y) ** 2) 
	error + jnp.sum(w1 ** 2) + jnp.sum(w2 ** 2)
	return error

loss_derivative = grad(loss_function, argnums=[2, 3, 4, 5])

n_epochs = 1000
eta = 0.005
p_bar = tqdm(range(n_epochs))
for i in p_bar:
	
	error = loss_function(x_inputs, y_classes, weights1, weights2, bias1, bias2)
	if error < mse_limit:
		break

	# Backward pass
	gradients = loss_derivative(x_inputs, y_classes, weights1, weights2, bias1, bias2)
	weights1 = weights1 - eta * gradients[0]
	bias1 = bias1 - eta * gradients[2]
	weights2 = weights2 - eta * gradients[1]
	bias2 = bias2 - eta * gradients[3]
	
	p_bar.set_description(f"Error: {error:.4f}")

y_pred = jnp.round(mlp(x_inputs, weights1, weights2, bias1, bias2))

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create figure

fig = make_subplots(rows=1, cols=2, subplot_titles=("XOR classification", "XOR classification"))
# Plot the original XOR problem
fig.add_trace(go.Scatter(x=x_inputs[y_classes == 0, 0], y=x_inputs[y_classes == 0, 1], mode='markers', name='Class 0', marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=x_inputs[y_classes == 1, 0], y=x_inputs[y_classes == 1, 1], mode='markers', name='Class 1', marker=dict(color='red')), row=1, col=1)
# Plot the XOR classification
fig.add_trace(go.Scatter(x=x_inputs[y_pred == 0, 0], y=x_inputs[y_pred == 0, 1], mode='markers', name='Class 0', marker=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=x_inputs[y_pred == 1, 0], y=x_inputs[y_pred == 1, 1], mode='markers', name='Class 1', marker=dict(color='red')), row=1, col=2)

# Add the decision boundary
x1 = np.linspace(-1.5, 1.5, 100)
x2 = np.linspace(-1.5, 1.5, 100)
X1, X2 = np.meshgrid(x1, x2)
X = np.array([X1.flatten(), X2.flatten()]).T
Z = mlp(X, weights1, weights2, bias1, bias2).reshape(100, 100)

fig.add_trace(go.Contour(x=x1, y=x2, z=Z, colorscale='Viridis', showscale=False), row=1, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="X1", row=1, col=1)
fig.update_xaxes(title_text="X1", row=1, col=2)
# Update yaxis properties
fig.update_yaxes(title_text="X2", row=1, col=1)
fig.update_yaxes(title_text="X2", row=1, col=2)
# Update title and size
fig.show()

Error: 5.9906:  24%|██▎       | 235/1000 [00:00<00:02, 354.52it/s] 

Error: 0.6385: 100%|██████████| 1000/1000 [00:03<00:00, 299.84it/s]


***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.3 - Understanding properties of our implementation

> 4. Perform multiple re-runs of the learning procedure (re-launching with different initializations)
>     1. What observations can you make on the learning process?
>     2. What happens if you initialize all weights to zeros?
>     3. Change your initialization and regularization scheme

</div>

***

When re-running the learning process mutliple times, we can see that the output
is extremely conditional on the initial position. When trying different methods
for initialization, we can see that sometimes the model can find local optima where 
it wraps around only a single point cluster.

When initializing with 0s, nothing happens, as there is no gradient to learn.

When changing the activation function, the model doesn't work anymore if a relu
is used in the last layer as the output is no longer regularized and is unbounded.
As such, it is fine in intermediate layers, but not in the final layer.

Regularization works somewhat to help make functions smaller, but the default
functionality is good enough as is.

In [ ]:

######################
# YOUR CODE GOES HERE
######################


***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Going further

> The following set of questions are optionnal addendum to your previous code that allow to understand more in-depth properties about _regularization_ both for neural networks and optimization in general.

> 1. (Optional) Implement the *weight decay* constraint in your network.
> 2. (Optional) Add the *momentum* to the learning procedure.

> *Weight decay* constraint
> As nothing constrains the weights in the network, we can note that usually all weights vector given a multiplicative factor might be equivalent, which can stall the learning (and lead to exploding weights). The *weight decay* allows to regularize the learning by penalizing weights with a too wide amplitude. The idea is to add this constraint as a term to the final loss (which leads to an indirect "pressure" on the learning process. Therefore, the final loss will be defined as
> $$
\begin{equation}
\mathcal{L}_{final}=\mathcal{L_D} + \lambda \sum_{l} \sum_{i} \sum_{j} \left( W_{ij}^{l} \right)^{2}
\end{equation}
$$
> where the parameter $\lambda$ controls the relative importance of the two terms.

> *Momentum* in learning
> Usually, in complex problems, the gradient can be very noisy and, therefore, the learning might oscillate widely. In order to reduce this problem, we can *smooth* the different gradient updates by retaining the values of the gradient at each iteration and then performing an update based on the latest gradient $\delta_{i}^{t}$ and the gradient at the previous iteration $\delta_{i}^{t-1}$. Therefore, a gradient update is applied as
> $$
\begin{equation}
\delta_{final}^{t} = \delta_{i}^{t} + m.\delta_{i}^{t-1}
\end{equation}
$$
> with $m$ the momentum parameter, which control the amount of gradient smoothing.

</div>

***

In [ ]:

######################
# YOUR CODE GOES HERE
######################


## (Optional) 3-layer audio classification

Once again, note that the following paragraph and subsequent questions are _optional_ but should be a quite simple extension of our previous work on 2-layer networks. If you struggle with the data import mechanisms, you can look down at the PyTorch section (with a **mandatory exercise**) that will provide base code for this aspect. 

Finally, we will attack a complete audio classification problem and try to perform neural network learning on a set of audio files. The data structure will be the same as the one used for parts 1 and 2. As discussed during the courses, even though a 2-layer neural network can provide non-linear boundaries, it can not perform "holes" inside those regions. In order to obtain an improved classification, we will now rely on a 3-layer neural network. The modification to the code of section 3.2 should be minimal, as the back-propagation will be similar for the new layer as one of the two others. We do not develop the math here as it is simply a re-application of the previous rules with an additional layer (which derivatives you should have generalized in the previous exercise).  

However, up until now, we only performed *binary classification* problems, but this time we need to obtain a decision rule for multiple classes. Therefore, we cannot rely on simply computing the distance between desired patterns and the obtained binary value. The idea here is to rely on the *softmax regression*, by considering classes as a vector of probabilities. The desired answers will therefore be considered as a set of *probabilities*, where the desired class is $1$ and the others are $0$ (called *one-hot* representation). Then, the cost function will rely on the softmax formulation

$$
\begin{equation}
\mathcal{L_D}(\theta) = - \frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{k} 1\left\{y^{(i)} = j\right\} \log \frac{e^{\theta_{j}^{T} x^{(i)}}}{\sum_{l=1}^{k} e^{ \theta_{l}^{T} x^{(i)} }}  \right]
\end{equation}
$$

Therefore, we compute the output of the softmax by taking 

$$
\begin{equation}
p(y^{(i)} = j | x^{(i)}; \theta) = \frac{e^{\theta_{j}^{T} x^{(i)}}}{\sum_{l=1}^{k} e^{ \theta_{l}^{T} x^{(i)}} }
\end{equation}
$$

By taking derivatives, we can show that the gradient of the softmax layer is

$$
\begin{equation}
\nabla_{\theta_{j}} \mathcal{L_D}(\theta) = - \frac{1}{m} \sum_{i=1}^{m}{ \left[ x^{(i)} \left( 1\{ y^{(i)} = j\}  - p(y^{(i)} = j \mid x^{(i)}, \theta) \right) \right]}
\end{equation}
$$

### Sweet activation functions

As discussed in the course, the interest of stacking layers is that there is an _activation function_, which allows non-linear interactions between the dimensions (and avoids to only compute a single huge affine transform). Although the `sigmoid` function has been historically the most used, there has been some large developments since. Notably the `ReLU` (Rectified Linear Unit) is one of the major difference in modern networks (we will see more about that in a later course)

In [78]:
# Function for computing the Sigmoid activation
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
# Derivative
def dsigmoid(a):
    return a * (1.0 - a)
# Function for computing the ReLU activation
def relu(x):
    return np.maximum(0, x)
# Derivative
def drelu(x):
    if (x < 0):
        return 0
    return 1
# Function for computing the Tanh activation
def tanh(x):
    return np.tanh(x);
# Derivative
def dtanh(x): 
    return np.cosh(x) ^ -2

Here, we plot some simple examples of what these activation functions look like. You can try to rely on these functions in your previous training code and witness the differences in training.


In [79]:
from bokeh.models import Div
from bokeh.layouts import column, row
from cml.plot import cml_figure
# Functions to
funcs = [('Sigmoid',sigmoid,'red'), ('Tanh',tanh,'orange'), ('ReLU',relu,'yellow')]
# Generating the x axis
x = np.linspace(-5, 5, 100)
plots = []
for (name, func, color) in funcs:
    cur_plot = cml_figure(plot_width=400, plot_height=250, title=name)
    cur_plot.line(x, func(x), color=color, line_width=4)
    plots.append(cur_plot)
plot = center_plot(column(Div(text = "Different activation functions", style={'font-size': '250%'}), row(*plots)))
plot.show()

NameError: name 'center_plot' is not defined

## Coding the whole network from scratch

You should now have all the tools necessary to apply neural networks from scratch to a more complex problem. In the following exercise, we simply removed any guideline code, and you need to code all the procedure for training a NN and **apply it to audio data**. You will use the spectral features discussed in the previous exercise as an input.

***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Going further **(optional exercise) **

>  1. Based on the previous neural network, upgrade the code to a 3-layer neural network
>  2. Implement the *softmax regression* on top of your 3-layer network
>  3. Use the provided code to perform classification on a pre-defined set of features
>  4. As previously, change the set of features to assess their different accuracies
>  5. Evaluate the neural network accuracy for all features combinations
>  6. What happens if the learning rate is too large ? What is this phenomenon ?
>  7. Perform a more advanced visualization of the learning process.
  

</div>

***

In [ ]:

######################
# YOUR CODE GOES HERE
######################


# Using `Pytorch` to enjoy life

Up to now, we have been writing every operations by ourselves (in order to better understand the mathematics behind NN). However, there exists of course some simplifying libraries that provide large simplifications to this question.

One of the most powerful and complete library of this sort is `Pytorch`, which has been developed for several years (even prior to the recent boom of deep learning). `Pytorch` provides a large set of pre-coded layers, but also **computational graphs** and **autograd**, which are very powerful paradigms allowing to define complex operators and automatically taking derivatives.

## An (extremely) fast and dirty introduction to `Pytorch`

PyTorch is a popular open-source deep learning framework based on the Torch library. It is primarily developed by Facebook AI research team, and it provides a simple and efficient way to build deep learning models. In this extremely short crash course, we will cover the basics of PyTorch, including:

* Tensors
* Automatic differentiation
* Neural networks
* Training a neural network on a dataset

For a complete tutorial on PyTorch, please check out the [official PyTorch documentation](https://pytorch.org/tutorials/). This tutorial covers a wide range of topics, including the basics of PyTorch, building and training neural networks, working with data and dataloaders, and deploying models to production.

In [1]:
import torch
import torch.nn as nn
x = torch.ones((100, 3))
net = nn.Sequential(
  nn.Linear(3, 20),
  nn.ReLU(),
  nn.Linear(20, 1),
  nn.Sigmoid()
)
l1 = nn.Linear(3, 20)
o1 = l1(x)
r1 = nn.ReLU()
o2 = r1(o1)
l2 = nn.Linear(20, 1)
s2 = nn.Sigmoid()
y = s2(l2(o2))
print(y.shape)


torch.Size([100, 1])


### Tensors
At its core, PyTorch is all about tensors. A tensor is a generalization of vectors and matrices to an arbitrary number of dimensions. A scalar is a 0-dimensional tensor, a vector is a 1-dimensional tensor, and a matrix is a 2-dimensional tensor.

#### Creating Tensors
We can create a PyTorch tensor from a Python list or a NumPy array using the `torch.tensor()` function, but also tensors of a specific size and/or with all zeros or all ones using the `torch.zeros()` and `torch.ones()` functions

In [2]:
import torch
import numpy as np
# create a tensor from a Python list
x = torch.tensor([1, 2, 3, 4])
print(x)
# create a tensor from a NumPy array
y = torch.tensor(np.array([[1, 2], [3, 4]]))
print(y)
# create a 2x3 tensor of zeros
z = torch.zeros((2, 3))
print(z)
# create a 3x2 tensor of ones
w = torch.ones((3, 2))
print(w)

tensor([1, 2, 3, 4])
tensor([[1, 2],
        [3, 4]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


#### Tensor operations

PyTorch tensors support a wide range of operations, including arithmetic operations like addition, subtraction, multiplication, and division, as well as more advanced operations like matrix multiplication, element-wise multiplication, and concatenation.

In [3]:
# create two tensors
x = torch.tensor([1, 2, 3, 4])
y = torch.tensor([5, 6, 7, 8])
# add the two tensors
z = x + y
print(z)
# element-wise multiplication
w = x * y
print(w)
# matrix multiplication
a = torch.tensor([[1, 2], [3, 4]])
b = torch.tensor([[5, 6], [7, 8]])
c = torch.matmul(a, b)
print(c)
# concatenation
d = torch.cat((a, b), dim=1)
print(d)

tensor([ 6,  8, 10, 12])
tensor([ 5, 12, 21, 32])
tensor([[19, 22],
        [43, 50]])
tensor([[1, 2, 5, 6],
        [3, 4, 7, 8]])


### Automatic differentiation

The core of PyTorch is the autograd package. It provides automatic differentiation for all operations on Tensors.

PyTorch's autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backpropagation is defined by how your code is run, and that every single iteration can be different.

To compute gradients, the `requires_grad` property of Tensors needs to `True` (which is the case by default). This tells PyTorch to track all operations on the Tensor. You can then call `.backward()` on a Tensor to compute the gradients with respect to that Tensor.

In [4]:
import torch

x = torch.tensor(2.0, requires_grad=True)
y = 2*x**2 + 3*x - 1
y.backward()

print(x.grad)
x.grad

tensor(11.)


tensor(11.)

## Defining our network

When building neural networks we frequently think of arranging the computation into layers, some of which have learnable parameters which will be optimized during learning. In `PyTorch`, the `nn` package provides higher-level abstractions over raw computational graphs that are useful for building neural networks. The `nn` package defines a set of `Modules`, which are roughly equivalent to neural network layers. A `Module` receives input `Tensors` and computes output `Tensors`, but may also hold internal state such as `Tensors` containing learnable parameters. The nn package also defines a set of useful loss functions that are commonly used when training neural networks.

In the following example, we use the `nn` package to show how easy it is to instantiate our previous three-layers network

In [5]:
import torch
# Define the input dimensions
in_size = 1000
# Number of neurons in a layer
hidden_size = 100
# Output (target) dimension
output_size = 10
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(in_size, hidden_size),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_size, hidden_size),
    torch.nn.Tanh(),
    torch.nn.Linear(hidden_size, output_size),
    torch.nn.Softmax(dim=1)
)

### Optimizing the network

Up to this point we have updated the weights of our models by manually performing the gradient descent algorithm (changing the parameters vectors). Although this is not a huge burden for simple optimization algorithms like stochastic gradient descent, in practice we often train neural networks using more sophisticated optimizers like AdaGrad, RMSProp or Adam (that we will see later in this course)

The `optim` package in PyTorch abstracts the idea of an optimization algorithm and provides implementations of commonly used optimization algorithms, and greatly simplfies the training loop associated with training a neural network.

For the sake of presentation we will use random inputs $\mathbf{x}$ that should be matched with random outputs $\mathbf{y}$

In [75]:
batch_size = 64
# Create random Tensors to hold inputs and outputs
x = torch.randn(batch_size, in_size)
y = torch.randn(batch_size, output_size)

class MyLayer(nn.Module):

    def forward()

SyntaxError: expected ':' (2193733760.py, line 8)

In the following example we optimize the model using the Adam algorithm provided by the `optim` package, based on a `MSE` loss.

In [76]:
# Learning rate
learning_rate = 1e-4
# Loss function that we will use
loss_fn = torch.nn.MSELoss(reduction='sum')
# Optimizer to fit the weights of the network
optimizer = torch.optim.RMSProp(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)
    # Compute the loss.
    loss = loss_fn(y_pred, y)
    # Before the backward pass, zero all of the network gradients
    optimizer.zero_grad()
    # Backward pass: compute gradient of the loss with respect to parameters
    loss.backward()
    # Calling the step function to update the parameters
    optimizer.step()
print(loss)
    

AttributeError: module 'torch.optim' has no attribute 'RMSProp'

## Exercise 2 - Pytorch to classify audio

Now that we know the main components of `Pytorch` to define and optimize networks, your assignement is to define a complete classification problem from audio data, by relying on this toolbox

***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.1 - Model and definition

>   1. Use `Pytorch` to define a model for audio classification
>   2. Import the audio features dataset and check that your model produces an output

</div>

***


In [12]:
import numpy as np
import librosa
from tqdm import tqdm 

def compute_transform(cur_signal, name, sr=44100, verbose = False):
    # Overall settings
    cur_signal = np.array(cur_signal)
    fSize = 1024
    wSize = fSize
    hSize = fSize//4
    refSr = 44100
    # Constant-Q settings
    fMin = librosa.note_to_hz('C2')
    nBins = 60 * 2
    import warnings
    warnings.filterwarnings('ignore')
    # Perform an analysis of spectral transform for each
    if (name == "stft"):
        # Compute the FFT 
        psc = librosa.stft(y=cur_signal, n_fft=fSize, win_length=wSize, hop_length=hSize, window='blackman')
        powerspec, phasespec = librosa.magphase(psc);
        return powerspec[:(fSize//2), :]
    elif (name == "mel"):
        # Compute the mel spectrogram        
        wMel = librosa.feature.melspectrogram(y=cur_signal, sr=sr, n_fft=fSize, hop_length=hSize)
        return wMel
    elif (name == "chroma"):
        # Compute the chromagram
        psc = librosa.stft(y=cur_signal, n_fft=fSize, win_length=wSize, hop_length=hSize, window='blackman')
        powerspec, phasespec = librosa.magphase(psc);
        wChroma = librosa.feature.chroma_stft(S=powerspec**2, sr=sr)
        return (wChroma)
    elif (name == "cqt"):
        # Compute the Constant-Q transform
        Xcq = librosa.cqt(y=cur_signal, sr=sr, n_bins=nBins, fmin=fMin, bins_per_octave=12 * 2)
        return (np.abs(Xcq));
    else:
        raise NotImplementedError("Audio descriptor " + name + " not implemented")

#
# Main features computation function
#
def compute_feature(cur_signal, name, verbose = False):
    # Window sizes
    cur_signal = np.array(cur_signal)
    wSize = 1024
    if (name == "loudness"):
        return librosa.feature.rms(y = cur_signal)
    # Compute the spectral centroid. [y, sr, S, n_fft, ...]
    elif name == "centroid":
        return librosa.feature.spectral_centroid(y = cur_signal)
        # Compute the spectral bandwidth. [y, sr, S, n_fft, ...]
    elif name == "bandwidth":
        return (librosa.feature.spectral_bandwidth(y = cur_signal))
        # Compute spectral contrast [R16] , sr, S, n_fft, ...])	
    elif name == "contrast":
        return (librosa.feature.spectral_contrast(y = cur_signal))
        # Compute the spectral flatness. [y, sr, S, n_fft, ...]
    elif name == "flatness":
        return (librosa.feature.spectral_flatness(y = cur_signal))
        # Compute roll-off frequency
    elif name == "rolloff":
        return (librosa.feature.spectral_rolloff(y = cur_signal))
    else:
        raise NotImplementedError("Audio descriptor " + name + " not implemented")

# import audio features dataset
import pandas as pd

from cml.data import import_dataset, AudioSupervisedDataset
data_path = "data/"
batch_size = 1
# Instantiate the dataset class
train_dataset = import_dataset(data_path, "musclefish", 'train', batch_size=batch_size)
# Generate a batch of data from the train dataset

df = pd.DataFrame(columns=["raw", "stft", "mel", "chroma", "cqt", "loudness", "centroid", "bandwidth", "contrast", "flatness", "rolloff", "y"])
for i, (data, label) in tqdm(enumerate(train_dataset)):
    features = [
        data.numpy().squeeze().astype(np.float32),
        compute_transform(data, "stft").squeeze().astype(np.float32),
        compute_transform(data, "mel").squeeze().astype(np.float32),
        compute_transform(data, "chroma").squeeze().astype(np.float32),
        compute_transform(data, "cqt").squeeze().astype(np.float32),
        compute_feature(data, "loudness").squeeze().astype(np.float32),
        compute_feature(data, "centroid").squeeze().astype(np.float32),
        compute_feature(data, "bandwidth").squeeze().astype(np.float32),
        compute_feature(data, "contrast").reshape(-1).astype(np.float32),
        compute_feature(data, "flatness").squeeze().astype(np.float32),
        compute_feature(data, "rolloff").squeeze().astype(np.float32),
        label.numpy()[0]
    ]

    df.loc[i] = features

df.shape

Recursive search in data//musclefish


820it [01:49,  6.89it/s]2024-05-09 11:09:29.732932: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
820it [01:49,  7.50it/s]


(820, 12)

In [20]:
torch.tensor(df["raw"][0])

tensor([ 2.4494e-07,  2.0210e-07,  9.4762e-08,  ..., -3.3393e-03,
        -1.7752e-03, -6.4241e-04])

In [34]:
%load_ext autoreload
%autoreload 2

######################
# YOUR CODE GOES HERE
######################


import torch.nn as nn

class AudioModel(nn.Module):
	def __init__(self):
		super(AudioModel, self).__init__()

		# # Convolution on the STFT (shape: 512, 173)
		# self.conv1 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=(1, 1))
		# self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# self.fcsftp = nn.Linear(8192, 32)
		# # Convolution on the Mel spectrogram (shape: 128, 173)
		# self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=(1, 1))
		# self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# self.fcMel = nn.Linear(128 * 173, 32)
		# # Convolution on the Chroma (shape: 12, 173)
		# self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=(1, 1))
		# self.pool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# self.fcChroma = nn.Linear(12 * 173, 32)
		# # Convolution on the CQT (shape: 120, 87)
		# self.conv4 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=(1, 1))
		# self.pool4 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# self.fcCQT = nn.Linear(120 * 87, 32)

		# Fully connected layers for features
		self.fc1 = nn.Linear(87, 32) # loudness (shape: 1, 87)
		self.fc2 = nn.Linear(87, 32) # centroid (shape: 1, 87)
		self.fc3 = nn.Linear(87, 32) # bandwidth (shape: 1, 87)
		self.fc4 = nn.Linear(87 * 7, 32) # contrast (shape: 7, 87)
		self.fc5 = nn.Linear(87, 32) # flatness (shape: 1, 87)
		self.fc6 = nn.Linear(87, 32) # rolloff (shape: 1, 87)

		# Fully connected layers
		self.fc7 = nn.Linear(32 * 6, 64)
		self.fc8 = nn.Linear(64, 64)
		self.fc9 = nn.Linear(64, 16)

		self.relu = nn.ReLU()
		self.softmax = nn.Softmax(dim=1)

	def forward(self, x):
		# stft.append(compute_transform(xx, "stft"))
		# mel.append(compute_transform(xx, "mel"))
		# chroma.append(compute_transform(xx, "chroma"))
		# cqt.append(compute_transform(xx, "cqt"))

		loudness = torch.tensor(x["loudness"])
		centroid = torch.tensor(x["centroid"])
		bandwidth = torch.tensor(x["bandwidth"])
		contrast = torch.tensor(x["contrast"])
		flatness = torch.tensor(x["flatness"])
		rolloff = torch.tensor(x["rolloff"])

		# stft = torch.tensor(stft)[None, :]
		# mel = torch.tensor(mel)[None, :]
		# chroma = torch.tensor(chroma)[None, :]
		# cqt = torch.tensor(cqt)[None, :]

		# print(
		# 	# stft.shape,
		# 	# mel.shape,
		# 	# chroma.shape,
		# 	# cqt.shape,
		# 	loudness.shape,
		# 	centroid.shape,
		# 	bandwidth.shape,
		# 	contrast.shape,
		# 	flatness.shape,
		# 	rolloff.shape
		# )

		# # Convolution on the STFT
		# stft = self.conv1(stft)
		# stft = self.pool1(stft)
		# stft = self.relu(stft)
		# stft = self.fcsftp(stft)
		# stft = self.relu(stft)
		
		# # Convolution on the Mel spectrogram
		# mel = self.conv2(mel)
		# mel = self.pool2(mel)
		# mel = self.relu(mel)
		# mel = self.fcMel(mel)
		# mel = self.relu(mel)

		# # Convolution on the Chroma
		# chroma = self.conv3(chroma)
		# chroma = self.pool3(chroma)
		# chroma = self.relu(chroma)
		# chroma = self.fcChroma(chroma)
		# chroma = self.relu(chroma)

		# # Convolution on the CQT
		# cqt = self.conv4(cqt)
		# cqt = self.pool4(cqt)
		# cqt = self.relu(cqt)
		# cqt = self.fcCQT(cqt)
		# cqt = self.relu(cqt)

		# Fully connected layers for features
		loudness = self.fc1(loudness)
		loudness = self.relu(loudness)
		centroid = self.fc2(centroid)
		centroid = self.relu(centroid)
		bandwidth = self.fc3(bandwidth)
		bandwidth = self.relu(bandwidth)
		contrast = self.fc4(contrast)
		contrast = self.relu(contrast)
		flatness = self.fc5(flatness)
		flatness = self.relu(flatness)
		rolloff = self.fc6(rolloff)
		rolloff = self.relu(rolloff)

		# Concatenate all features

		# print(
		# 	# stft.shape,
		# 	# mel.shape,
		# 	# chroma.shape,
		# 	# cqt.shape,
		# 	loudness.shape,
		# 	centroid.shape,
		# 	bandwidth.shape,
		# 	contrast.shape,
		# 	flatness.shape,
		# 	rolloff.shape
		# )
		x = torch.cat((
			# stft,
			# mel, 
			# chroma, 
			# cqt, 
			loudness,
			centroid, 
			bandwidth, 
			contrast, 
			flatness, 
			rolloff
			))

		# print(x.shape)

		# Fully connected layers
		x = self.fc7(x)
		x = self.relu(x)
		x = self.fc8(x)
		x = self.relu(x)
		x = self.fc9(x)
		x = self.softmax(x[None, :])

		return x.reshape(-1, 16)

In [7]:
import torch.nn as nn

class AudioModelSimple(nn.Module):
	def __init__(self):
		super(AudioModelSimple, self).__init__()
		# Use simple statistical measures from the features
		# Features: loudness, centroid, bandwidth, contrast, flatness, rolloff
		# stats = ["mean", "std", "min", "max"]

		# Fully connected layers for features
		self.fc1 = nn.Linear(4 * 6, 32)
		self.fc2 = nn.Linear(32, 32)
		self.fc3 = nn.Linear(32, 32)
		self.fc4 = nn.Linear(32, 16)

		self.relu = nn.ReLU()
		self.softmax = nn.Softmax(dim=1)

	def forward(self, x):
		x = torch.tensor([
			x["loudness"].mean(),
			x["loudness"].std(),
			x["loudness"].min(),
			x["loudness"].max(),
			x["centroid"].mean(),
			x["centroid"].std(),
			x["centroid"].min(),
			x["centroid"].max(),
			x["bandwidth"].mean(),
			x["bandwidth"].std(),
			x["bandwidth"].min(),
			x["bandwidth"].max(),
			x["contrast"].mean(),
			x["contrast"].std(),
			x["contrast"].min(),
			x["contrast"].max(),
			x["flatness"].mean(),
			x["flatness"].std(),
			x["flatness"].min(),
			x["flatness"].max(),
			x["rolloff"].mean(),
			x["rolloff"].std(),
			x["rolloff"].min(),
			x["rolloff"].max()
		])

		# Fully connected layers for features
		x = self.fc1(x)
		x = self.relu(x)
		x = self.fc2(x)
		x = self.relu(x)
		x = self.fc3(x)
		x = self.relu(x)
		x = self.fc4(x)
		x = self.softmax(x[None, :])

		return x.reshape(-1, 16)

In [29]:
import torch.nn as nn

class AudioModelConv(nn.Module):
    def __init__(self):
        super(AudioModelConv, self).__init__()

        # Convolution on the raw signal
        self.conv1 = nn.Conv1d(1, 32, kernel_size=9, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=9, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=9, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5507, 128)
        self.fc2 = nn.Linear(128, 16)

        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = torch.tensor(x["raw"], dtype=torch.float32)[None, :]
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.relu(x)
        x = self.flatten(x)[None, :]
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)

        return x


***

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.2 - Training the model

>   3. Write the optimization loop (think carefully about the _loss function_
>   4. As previously, change the set of features to assess their different accuracies
>   5. (Optional) Think of how you could use more complex features (time series, audio, STFT) to classify your data

</div>

***

In [36]:
######################
# YOUR CODE GOES HERE
######################
import torch
import tensorflow as tf
import torch.optim as optim
from tqdm import tqdm

# Instantiate the model
model = AudioModel()
# model = AudioModelSimple()
# model = AudioModelConv()
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Shuffle and split the dataset
df = df.sample(frac=1).reset_index(drop=True)
train_df = df.iloc[:int(0.8 * len(df))]
test_df = df.iloc[int(0.8 * len(df)):]

# Number of epochs
n_epochs = 100

# Training loop
p_bar = tqdm(range(n_epochs))
for epoch in p_bar:
	running_loss = 0.0
	train_df = train_df.sample(frac=1).reset_index(drop=True)
	for i, data in train_df.iterrows():
		# get the inputs; data is a list of [inputs, labels]
		inputs, labels = data[:-1], data[-1]       
		# zero the parameter gradients
		optimizer.zero_grad()
		# forward + backward + optimize
		outputs = model(inputs)
		# onehot encode labels
		labels = torch.tensor(labels, dtype=torch.long)
		labels = torch.nn.functional.one_hot(labels, num_classes=16).type(torch.float32)[None, :]
		# print(labels.dtype, outputs.dtype)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()
		# print statistics
		running_loss += loss.item()
	p_bar.set_description(f"Loss: {running_loss:.4f}")

print('Finished Training')

# Test the model
correct = 0
total = 0

with torch.no_grad():
	for i, data in test_df.iterrows():
		inputs, labels = data[:-1], data[-1]
		outputs = model(inputs)
		labels = torch.tensor(labels, dtype=torch.long)
		labels = torch.nn.functional.one_hot(labels, num_classes=16).type(torch.float32)[None, :]
		_, predicted = torch.max(outputs, 1)
		_, labels = torch.max(labels, 1)
		total += 1
		correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))


Loss: 1729.7357: 100%|██████████| 100/100 [06:02<00:00,  3.63s/it]

Finished Training
Accuracy of the network on the test data: 25 %


# Conclusion

Honnestly, I tried a lot of stuff. I didn't really want to copy an online tutorial,
but I really just wanted to have some fun with it. It turns out my model works 
very badly, but I don't want to work on this project for longer than I already
have (easily 5 hours of debugging, training and testing architectures).

## Idea

I was initally very curious if it is possible to combine multiple model types into
a compound model. From previous courses, I knew I could potentially get good results 
with simple things such as a 4-5 layer MLP that predicts on simple parameters and 
some hyperparameter tuning, but this time, I tried to play around a bit more with 
architectures. 

In papers and modern techniques, it is quite often seen that models take a single 
tensor and predict on that. While it is more computationally sensible for large
data models, with this small dataset I tried to go for a more patchwork approach.

The idea was to tranform the input sound-vector into it's feature and transforms,
and then use a kind of agreegate model to make predictions. The issue with this 
approach in this case has been the small dataset. Combined models can sometimes 
work, especially if you know what features to use, and don't want the exploding 
amount of weights you have with concatenanting all the features together, so I 
just wanted to try it out.

## End

Either way, in the end, I wasted a long time trying a lot of stuff that didn't
work, so I am going to hand it in in this unaltered state. I hope the work I did 
is enough despite the poor performance.